In [ ]:
# 回测（低级 API）

[NautilusTrader](https://nautilustrader.io/docs/) 高性能算法交易平台和事件驱动回测器教程。

[在 GitHub 上查看源码](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/getting_started/backtest_low_level.ipynb)。


In [ ]:
## 概述

本教程将介绍如何使用 `BacktestEngine` 在模拟币安现货交易所上使用历史交易 tick 数据，结合 TWAP 执行算法回测简单的 EMA 交叉策略。

将涵盖以下要点：
- 如何使用数据加载器和整理器加载原始数据（Nautilus 外部）
- 如何将此数据添加到 `BacktestEngine`
- 如何向 `BacktestEngine` 添加场所、策略和执行算法
- 如何使用 `BacktestEngine` 运行回测
- 运行后分析和重复运行选项


In [ ]:
## 先决条件
- 已安装 Python 3.11+
- 已安装 [JupyterLab](https://jupyter.org/) 或类似软件 (`pip install -U jupyterlab`)
- 已安装 [NautilusTrader](https://pypi.org/project/nautilus_trader/) 最新版本 (`pip install -U nautilus_trader`)


In [ ]:
## 导入

我们将从本教程其余部分的所有导入开始。


In [ ]:
from decimal import Decimal

from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.examples.algorithms.twap import TWAPExecAlgorithm
from nautilus_trader.examples.strategies.ema_cross_twap import EMACrossTWAP
from nautilus_trader.examples.strategies.ema_cross_twap import EMACrossTWAPConfig
from nautilus_trader.model import BarType
from nautilus_trader.model import Money
from nautilus_trader.model import TraderId
from nautilus_trader.model import Venue
from nautilus_trader.model.currencies import ETH
from nautilus_trader.model.currencies import USDT
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.persistence.wranglers import TradeTickDataWrangler
from nautilus_trader.test_kit.providers import TestDataProvider
from nautilus_trader.test_kit.providers import TestInstrumentProvider


In [ ]:
## 加载数据

对于本教程，我们将使用 NautilusTrader 存储库中存在的一些存根测试数据
（此数据也被自动化测试套件用来测试平台的正确性）。

首先，实例化一个数据提供器，我们可以使用它将原始 CSV 交易 tick 数据读入内存作为 `pd.DataFrame`。
然后我们需要初始化与数据匹配的工具，在这种情况下是币安的 `ETHUSDT` 现货加密货币对。
我们将在本次回测运行的其余部分中使用此工具。

接下来，我们需要将此数据整理成 Nautilus `TradeTick` 对象的列表，稍后我们可以将其添加到 `BacktestEngine`。


In [ ]:
# 加载存根测试数据
provider = TestDataProvider()
trades_df = provider.read_csv_ticks("binance/ethusdt-trades.csv")

# 初始化与数据匹配的工具
ETHUSDT_BINANCE = TestInstrumentProvider.ethusdt_binance()

# 处理成 Nautilus 对象
wrangler = TradeTickDataWrangler(instrument=ETHUSDT_BINANCE)
ticks = wrangler.process(trades_df)


In [ ]:
有关典型数据处理组件和管道的更详细说明，请参阅[加载外部数据](https://nautilustrader.io/docs/latest/concepts/data#loading-data)指南。


In [ ]:
## 初始化回测引擎

现在我们需要一个回测引擎，最少你可以只调用 `BacktestEngine()`，它将实例化一个具有默认配置的引擎。

这里我们还展示初始化一个 `BacktestEngineConfig`（仅指定了自定义 `trader_id`）
来展示一般配置模式。

有关所有可用配置选项的详细信息，请参阅[配置](https://nautilustrader.io/docs/api_reference/config) API 参考。


In [ ]:
# 配置回测引擎
config = BacktestEngineConfig(trader_id=TraderId("BACKTESTER-001"))

# 构建回测引擎
engine = BacktestEngine(config=config)


In [ ]:
## 添加场所

我们需要一个交易场所，它应该与添加到引擎的*市场*数据匹配。

在这种情况下，我们将设置一个*模拟*币安现货交易所。


In [ ]:
# 添加交易场所（可以有多个场所）
BINANCE = Venue("BINANCE")
engine.add_venue(
    venue=BINANCE,
    oms_type=OmsType.NETTING,
    account_type=AccountType.CASH,  # 现货 CASH 账户（不适用于永续或期货）
    base_currency=None,  # 多币种账户
    starting_balances=[Money(1_000_000.0, USDT), Money(10.0, ETH)],
)


In [ ]:
## 添加数据

现在我们可以向回测引擎添加数据。首先添加我们之前初始化的与数据匹配的 `Instrument` 对象。

然后我们可以添加我们之前整理的交易。


In [ ]:
# 添加工具
engine.add_instrument(ETHUSDT_BINANCE)

# 添加数据
engine.add_data(ticks)


In [ ]:
:::note
数据的数量和数据类型的多样性仅受机器资源和您的想象力限制（自定义类型是可能的）。
此外，多个场所可以用于回测，同样仅受机器资源限制。
:::


In [ ]:
## 添加策略

现在我们可以添加我们想要作为系统一部分运行的交易策略。

:::note
多个策略和工具可以用于回测，仅受机器资源限制。
:::

首先，初始化策略配置，然后使用它来初始化我们可以添加到引擎的策略：


In [ ]:
# 配置您的策略
strategy_config = EMACrossTWAPConfig(
    instrument_id=ETHUSDT_BINANCE.id,
    bar_type=BarType.from_str("ETHUSDT.BINANCE-250-TICK-LAST-INTERNAL"),
    trade_size=Decimal("0.10"),
    fast_ema_period=10,
    slow_ema_period=20,
    twap_horizon_secs=10.0,
    twap_interval_secs=2.5,
)

# 实例化并添加您的策略
strategy = EMACrossTWAP(config=strategy_config)
engine.add_strategy(strategy=strategy)


In [ ]:
您可能注意到此策略配置包含与 TWAP 执行算法相关的参数。
这是因为我们可以灵活地在每个订单提交时使用不同的参数，我们仍然需要初始化
并添加实际的 `ExecAlgorithm` 组件，它将执行算法 - 我们现在就这样做。

## 添加执行算法

NautilusTrader 使我们能够构建非常复杂的自定义组件系统。这里我们只展示其中一个可用的自定义组件，
在这种情况下是内置的 TWAP 执行算法。它的配置和添加到引擎的模式通常与策略相同：

:::note
多个执行算法可以用于回测，仅受机器资源限制。
:::


In [ ]:
# 实例化并添加您的执行算法
exec_algorithm = TWAPExecAlgorithm()  # 使用默认值
engine.add_exec_algorithm(exec_algorithm)


In [ ]:
## 运行回测

现在我们已经配置了数据、场所和交易系统 - 我们可以运行回测
只需调用 `.run(...)` 方法，默认情况下将对所有可用数据运行回测。

有关所有可用方法和选项的完整描述，请参阅 [BacktestEngineConfig](https://nautilustrader.io/docs/latest/api_reference/config) API 参考。


In [ ]:
# 运行引擎（从数据开始到结束）
engine.run()


In [ ]:
## 运行后分析

一旦回测完成，将使用一些默认统计信息（或可以加载的自定义统计信息，请参阅高级[投资组合统计](../concepts/advanced/portfolio_statistics.md)指南）自动记录运行后的统计表。

此外，许多结果数据和执行对象将保存在内存中，我们可以使用它们通过生成各种报告来进一步分析性能。


In [ ]:
engine.trader.generate_account_report(BINANCE)


In [ ]:
engine.trader.generate_order_fills_report()


In [ ]:
engine.trader.generate_positions_report()


In [ ]:
## 重复运行

我们还可以选择重置引擎以使用不同的策略和组件配置进行重复运行。
调用 `.reset(...)` 方法将保留所有加载的数据和组件，但重置所有其他有状态值，
就像我们有一个新的 `BacktestEngine`（这避免了再次加载相同数据的需要）。


In [ ]:
# 对于重复的回测运行，确保重置引擎
engine.reset()


In [ ]:
各个组件（参与者、策略、执行算法）需要根据需要删除和添加。

有关实现这一点的所有可用方法的描述，请参阅[交易者](../api_reference/trading.md) API 参考。


In [ ]:
# 完成后，如果脚本继续，最好处理对象
engine.dispose()
